In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv

load_dotenv()

llm=HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task= "text-generation"
)

model= ChatHuggingFace(llm=llm)

C:\Users\Aritri Baidya\Desktop\MyFiles\Pycharm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#create a state
class BlogState(TypedDict):
    title: str
    outline : str
    content: str

In [4]:
def create_outline(state: BlogState) -> BlogState:
    #fetch title
    title=state['title']

    #call llm gen outline
    prompt=f'Generate a detailed outline for a blog on the topic {title}'
    outline=model.invoke(prompt).content

    #update state
    state['outline']=outline

    return state



In [5]:
def create_blog(state: BlogState) -> BlogState:
    #fetch title
    title=state['title']
    outline=state['outline']

    #call llm gen blog
    prompt=f'Write a detailed blog on the topic - {title} using the following outline- \n {outline}'
    content=model.invoke(prompt).content

    #update state
    state['content']=content

    return state

In [6]:
#create the graph
graph= StateGraph(BlogState)

#add nodes
graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)

#add edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog',END)

#compile
workflow=graph.compile()

In [7]:
#execute

initial_state={'title': 'Rise of AI in India'}

final_state=workflow.invoke(initial_state)

print(final_state)


{'title': 'Rise of AI in India', 'outline': 'I. Introduction\n   A. Brief introduction about AI and its significance in today\'s age\n   B. Importance of AI in India\'s economic and social development\n   C. Preview of the blog\'s structure\nII. Government initiatives and policies for AI development\n   A. National Programme on Artificial Intelligence (NPAI)\n   B. National Strategy for AI (NSAI)\n   C. Collaborations and partnerships\nIII. Impact of AI on Indian industries\n   A. Agriculture\n   B. Healthcare\n   C. Education\n   D. Banking and Finance\n   E. Manufacturing\nIV. AI and employment\n   A. Job creation and reskilling\n   B. Implications for the workforce\nV. AI and the future of work\nVI. Challenges and limitations of AI adoption in India\n   A. Data privacy and security concerns\n   B. Lack of talent and skills\n   C. Infrastructure and accessibility\n   D. Regulation and standardization\nVII. Future of AI in India\n   A. Opportunities and potential of AI\n   B. Role of 

In [8]:
print(final_state['outline'])

I. Introduction
   A. Brief introduction about AI and its significance in today's age
   B. Importance of AI in India's economic and social development
   C. Preview of the blog's structure
II. Government initiatives and policies for AI development
   A. National Programme on Artificial Intelligence (NPAI)
   B. National Strategy for AI (NSAI)
   C. Collaborations and partnerships
III. Impact of AI on Indian industries
   A. Agriculture
   B. Healthcare
   C. Education
   D. Banking and Finance
   E. Manufacturing
IV. AI and employment
   A. Job creation and reskilling
   B. Implications for the workforce
V. AI and the future of work
VI. Challenges and limitations of AI adoption in India
   A. Data privacy and security concerns
   B. Lack of talent and skills
   C. Infrastructure and accessibility
   D. Regulation and standardization
VII. Future of AI in India
   A. Opportunities and potential of AI
   B. Role of startups and entrepreneurs
VIII. Conclusion
   A. Importance and urgency 

In [9]:
print(final_state['content'])


In 2018, the Indian government launched the National Programme on Artificial Intelligence (NPAI) with an aim to build a robust AI ecosystem in the country. The programme focuses on AI research, education, and policy development. NPAI's vision is to make India a global hub for AI innovation and application by boosting AI talent, increasing AI startups, and creating AI solutions for social challenges. The programme aims to allocate INR 7,500 crore (approximately USD 1 billion) for the same and takes inspiration from the UK's governmental approach to AI. The NPAI's strategic interventions for research, development, and adoption of AI will lead to a 10x boost in the AI industry's size by 2025. By 2025, it intends to make India a top destination for AI research and talent, with the following objectives:

A. Encouraging research and development of AI: The NPAI focuses on supporting research and development in fundamental AI areas such as Natural Language Processing (NLP), Computer Vision, a

In [ ]:
#NOTE: In case of langchain it was not possible to see an intermediate function even when the same concept of chaining was present. In LangGraph it is possible because of the concept of 'State' which helps passing of information from every node to the next and also allows for updating the information being passed.